In [1029]:
import numpy as np
import random
import math
import collections
import operator
import time

In [1030]:
def generate_population(seed_children_dict):
    new_seeds = []
    for seed, nchildren in seed_children_dict.items():
        for _ in range(nchildren):
            x = seed[0]
            y = seed[1]
            new_x = random.gauss(x,1)
            new_y = random.gauss(y,1)
            
            while new_x < 0 or new_x > 10:
                new_x=random.gauss(x,1)
                
            while new_y < 0 or new_y > 10:
                new_y=random.gauss(y,1)
                    
            new_seeds.append((new_x, new_y))
        
    return new_seeds

In [1031]:
def get_fitness(seed_array):
    fitness_array = []
    for seed in seed_array:
        try:
            x = seed[0]
            y = seed[1]
            fitness = ( x * math.sin(4*x) ) + ( 1.1 * y * math.sin( 2 * y) )
        except ValueError:
            print(x,y, seed_array)
        fitness_array.append(fitness)
    return fitness_array

In [1032]:
def normalize_fitness(sorted_fitness_dict):
    lowest_fitness_element = list(sorted_fitness_dict.items())[0]
    if lowest_fitness_element:
        new_vals = []
        for val in sorted_fitness_dict.values():
            new_val = val - list(sorted_fitness_dict.values())[0]
            new_vals.append(new_val)
        sorted_fitness_dict = dict(zip(sorted_fitness_dict.keys(), new_vals))
    
    return sorted_fitness_dict
            


In [1092]:
def eval_fitness(normalized_fitness_dict):
    children_seed_list = []
    maximum = max(normalized_fitness_dict.values())
    for val in normalized_fitness_dict.values():
        num_seed = round( ( (val+0.000001) ) / maximum)
        children_seed_list.append(num_seed)
        
    seed_children_dict = dict(zip(normalized_fitness_dict.keys(), children_seed_list[::-1]))
        
    return seed_children_dict

In [1034]:
def get_new_seed_count(seed_children_dict):
    new_seeds_count = 0
    for seed, children in seed_children_dict.items():
        for i in range(children):
            new_seeds_count += children
            
    return new_seeds_count
    
            

In [1035]:
def prune_seeds(current_fitness_dict):
    sorted_current_fitness_dict = {k: v for k, v in sorted(current_fitness_dict.items(), key=lambda item: item[1])}
    #print(list(({k: sorted_current_fitness_dict[k] for k in list(sorted_current_fitness_dict)}.items()))[:5])
    current_fitness_dict = dict(list(sorted_current_fitness_dict.items())[0: 100]) 

In [1382]:
pmax = 10**2
maxiter = 10**2
delta_cap = 10**9
num_exceeded_delta = 6


seed_array = []
for i in range(100):
    x = random.uniform(0, 10)
    y = random.uniform(0, 10)
    seed_array.append((round(x, 3),round(y, 3)))


delta = delta_cap + 0.01
c = 0
current_fitness_dict = {}
best_fitness_list =  []


start_time = time.time()

for i in range(1, maxiter+1):
    fitness_array = get_fitness(seed_array)
    fitness_dict = dict(zip(seed_array, fitness_array))
    sorted_fitness_dict = {k: v for k, v in sorted(fitness_dict.items(), key=lambda item: item[1])}

    normalized_fitness_dict = normalize_fitness(sorted_fitness_dict)
    seed_children_dict = eval_fitness(normalized_fitness_dict)

    fitness_sum = sum(fitness_array)

    best_fitness_list.append(sorted(fitness_array)[0])



    for seed, fitness in fitness_dict.items():
        current_fitness_dict[seed] = fitness

    if len(current_fitness_dict) > pmax:
            prune_seeds(current_fitness_dict)



    seed_array = seed_array + generate_population(seed_children_dict)

    #print((best_fitness_list[-1]), best_fitness_list[-1])
    if len(best_fitness_list) > 5:
        delta = best_fitness_list[-1] - best_fitness_list[-2]
    else:
        delta = delta_cap + 0.01



    #print(f"Iteration: {i}\n")
    #print(f"\tSummed fitness: {fitness_sum}")
    #print(f"\tBest three fitness scores from this iteration: {sorted(fitness_array)[:3]}")
    #print(f"\tBest seed: {list(sorted_fitness_dict.items())[0]})")
    #print(f"\tNew seeds from this iteration based on fitness score: {get_new_seed_count(seed_children_dict)}")
    #print("\n\n")
    
    if delta < delta_cap:
        c += 1
    
    if c == num_exceeded_delta:
        #print(f"The difference between iterations was lower than the set delta value of {delta_cap}\n")
        break
        

      
end_time = time.time()
print(f"Loop runtime: {end_time - start_time}\n")
print("Last iteration:\n")
print(f"\tBest seed: {min(current_fitness_dict.items(), key=operator.itemgetter(1))[0]}")
print(f"\tBest fitness: {min(current_fitness_dict.items(), key=operator.itemgetter(1))[1]}")
print(f"\tBest three fitness scores: {best_fitness_list[::-1][:3]}")

Loop runtime: 2.0768113136291504

Last iteration:

	Best seed: (9.042, 8.654)
	Best fitness: -18.550211740481537
	Best three fitness scores: [-18.550211740481537, -18.550211740481537, -18.550211740481537]
